In [1]:
import os
import json
import fiona
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

from collections import Counter

from shapely.geometry import shape
from shapely.geometry import Point

# Gee and EE 
import ee

# Clustering (best for testing** due to speed)
from sklearn.cluster import KMeans

# tif file creation
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Plotting and Vis 
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import re

In [2]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [3]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [4]:
## TEMP REMOVE ADDITIONAL STUFF FOR QUICKER TESTING 
# tile_43641125_34108721_43915744_34336837 only 

chunks = chunks[chunks['file_name'] == 'tile_43641125_34108721_43915744_34336837']

In [5]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [6]:
def create_subregions(chunks, sift_percentage_lon=0.5, sift_percentage_lat=0.415):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount_lon = length_lon * sift_percentage_lon 
    shift_amount_lat = length_lon * sift_percentage_lat

    shift_directions = {
        "left": (-shift_amount_lon, 0),
        "right": (shift_amount_lon, 0),
        "up": (0, shift_amount_lat),
        "down": (0, -shift_amount_lat),
        "top_left": (-shift_amount_lon, shift_amount_lat),
        "top_right": (shift_amount_lon, shift_amount_lat),
        "bottom_left": (-shift_amount_lon, -shift_amount_lat),
        "bottom_right": (shift_amount_lon, -shift_amount_lat),
    }

    subregions = []
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

# Reference Points

In [7]:
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [8]:
# Example of 3 maps for urban, agricultrual, water and waste
urban_map = gen_basemap(basemap_url, aoi=chunks)
agricultural_map = gen_basemap(basemap_url, aoi=chunks)
water_map = gen_basemap(basemap_url, aoi=chunks)
wasteland_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_map = geemap.Map()

In [9]:
urban_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [10]:
agricultural_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# water_map = water_waste_map
water_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
wasteland_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
additional_map = gen_basemap(basemap_url, aoi=chunks)
additional_map


Map(center=[34.22282359393232, 43.778616017576226], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
additional_points = collect_points_from_geemap(additional_map, label="Urban")
additional_points

,geometry,label


In [15]:
# convert to labelled points in GeoDataFrame
water_points = collect_points_from_geemap(water_map, label="Water")
agricultural_points = collect_points_from_geemap(agricultural_map, label="Agricultural")
urban_points = collect_points_from_geemap(urban_map, label="Urban")
wasteland_map = collect_points_from_geemap(wasteland_map, label="Wasteland")

# merge into one GeoDataFrame
points_list = [water_points, agricultural_points, urban_points, wasteland_map]
labels = pd.concat(points_list, ignore_index=True)


# Clustering
Here we create a Sparse matrix

then with the sparse matrix compare and label based on that

In [16]:
pth = "data/"

# collect paths for all csv files in the folder
paths = []

for root, dirs, files in os.walk(pth):
    for file in files:
        if file.endswith(".csv"):
            paths.append(os.path.join(root, file))

print(f"Found {len(paths)} CSV files.")

# select 2024 
path = [p for p in paths if "2019" in p]
path


Found 6 CSV files.


['data/2019.csv']

In [17]:
def prep_data(dir):
    """Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact."""
    df1 = pd.read_csv(dir)
    df1 = df1.dropna()
    df2_clear = df1.copy()
    df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
    df2_clear = df2_clear.set_index("file_name")
    df1 = df1.set_index("file_name")
    df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

    return df1, df2_clear

df1, df2_clear = prep_data(path[0])

mapping = {
            None: 0,
            np.NaN: 0,
            "NaN": 0,
            "None": 0,
            "Water": 1,
            "Agricultural": 2,
            "Urban": 3,
            "Wasteland": 4,
        }

In [18]:
import cluster as cl 
from cluster import kmeans_clustering

In [19]:
cluster = cl.Cluster(chunks, df1, points=labels, mapping=mapping, passes=6)
cluster.reload_state(filname_prefix="temp_test/test3", filename_postfix="_state")

In [20]:
import ee
import geemap
from ipyleaflet import CircleMarker
import ipywidgets as widgets
import pandas as pd 

In [21]:
label_list = ['Urban', 'Agricultural', 'Water', 'Wasteland', 'Wetland', 'Other']
date_range = [i for i in range(2019, 2025)] 

collection_name = "COPERNICUS/S2_SR_HARMONIZED"

with fiona.open("aoi.geojson") as src:
    aoi = gpd.GeoDataFrame.from_features(src, crs=src.crs)
 

In [22]:
# empty label list 
labels = gpd.GeoDataFrame(columns=["label", "geometry", "start_year", "end_year"], geometry="geometry")

In [23]:
m = geemap.Map()

def map_polygon(polygon, collection_name, layer_name, yyyymmdd1="2024-01-01", yyyymmdd2="2024-12-29", num_tasks=10):

    # Load in Year Data to Aid in Label Creation
    collection = ee.ImageCollection(collection_name) \
        .filterDate(yyyymmdd1, yyyymmdd2) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
        .median() \
        .clip(polygon)
    
    vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

    m.addLayer(collection, vis, layer_name)

    # center map on the polygon
    coords = polygon.geometry().centroid().coordinates().getInfo()
    m.setCenter(coords[0], coords[1], 10)

label_dropdown = widgets.Dropdown(
    options=label_list,
    description='Label:',
    value=label_list[0],
)

# Plots Each Year
for year in date_range:
    map_polygon(geemap.geopandas_to_ee(aoi), collection_name, f"Year {year}", yyyymmdd1=f"{year}-01-01", yyyymmdd2=f"{year}-12-29")

date_slider = widgets.SelectionSlider(
    options=[str(year) for year in range(2019, 2025)], # 2019 to 2024 for full years
    value='2019',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

time_frame_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)], # default 12 as its the full year 
    value='3',
    description='timeframe:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

colour_dict = {
    "Urban": "blue",
    "Agricultural": "green",
    "Water": "cyan",
    "Wasteland": "brown",
    "Wetland": "lightblue", 
} # other is grey by default

def append_label_to_points(b):
    """Appends the selected label to the points in the labels GeoDataFrame."""

    global labels

    selected_label = label_dropdown.value
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1

    # Extract drawn features (assumed to be Points)
    all_drawn_features = m.draw_features
    if not all_drawn_features:
        print("No features drawn.")
        return
    
    points = []
    for feature in all_drawn_features:
        try:
            coords = feature.geometry().coordinates().getInfo()  # [lon, lat]
            point = Point(coords)
            points.append(point)
        except Exception as e:
            print("Skipping feature due to error:", e)

    # create dataframe from points
    new_labels = gpd.GeoDataFrame({
        'label': [selected_label] * len(points),
        'start_year': [start_year] * len(points),
        'end_year': [end_year] * len(points),
        'geometry': points
    }, crs="EPSG:4326")

    # Clear All Drawn Features
    m.draw_control.clear()


    # Append to the existing labels GeoDataFrame
    labels = pd.concat([labels, new_labels], ignore_index=True)
    

# Label Range  
label_output = widgets.Label()
def update_label(*args):
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1
    label_output.value = f"Selected range: {start_year} to {end_year}"

date_slider.observe(update_label, names='value')
time_frame_slider.observe(update_label, names='value')

update_label() 

# render_btn = widgets.Button(description="Draw Selected Date Layer", position="bottomright") 
# widget_draw = widgets.VBox([render_btn])
# m.add_widget(widget_draw, position="bottomright")

append_btn = widgets.Button(description="Append Label to Points")
append_btn.on_click(append_label_to_points)

# display(widgets.HBox([date_slider, month_slider, time_frame_slider, render_btn]))
display(widgets.VBox([
    widgets.HBox([date_slider, time_frame_slider]),
    widgets.HBox([label_dropdown, label_output, append_btn]),
]))

m

Map(center=[34.22411402596497, 43.91430818231603], controls=(WidgetControl(options=['position', 'transparent_b…

In [28]:
labels.head()

,label,geometry,start_year,end_year
0,Urban,POINT (43.88006 34.20145),2019,2024
1,Urban,POINT (43.88761 34.20457),2019,2024
2,Urban,POINT (43.85432 34.18292),2019,2024
3,Urban,POINT (43.90458 34.19124),2019,2024


In [29]:
data_dir = pd.DataFrame({
    "dir": [f"data/{year}.csv" for year in range(2019, 2025)],
    "year": [year for year in range(2019, 2025)]
})

In [30]:
class ts_cluster():

    cluster_list = []

    yearly_labels_dict = []
    common_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])
    
    data_dir = pd.DataFrame(columns=["dir", "year"])

    start_year = 2019
    end_year = 2024

    def __init__(self, ts_point_labels, data_dir, mapping, start_year, end_year, passes=6, full_data=True):
        """
            Initializes the ts_cluster object with time-series point labels, data directory, mapping, and number of passes.
        """
        
        self.start_year = int(start_year)
        self.end_year = int(end_year)
        self.full_data = full_data

        self.data_dir = data_dir

        if start_year > end_year:
            raise ValueError("start_year must be less than or equal to end_year")
        
        common_labels, uncommon_labels = self.__reduce_to_common__(ts_point_labels)
        self.common_labels = common_labels
        self.yearly_labels_dict = {
            year: self.__split_year__(year, uncommon_labels)
            for year in range(start_year, end_year + 1)
        }

        # runtime vars 
        self.mapping = mapping
        self.passes = passes

    def __reduce_to_year__(year, label_df):
        """
            Returns a DataFrame with only entries that overlap with the specified year.

        Args:
            year (int):                The year to filter the labels DataFrame.
            label_df (GeoDataFrame):   The labels DataFrame to filter.

        Returns:
            GeoDataFrame:              A DataFrame containing only the labels that overlap with the specified year.
        """

        year_df = label_df[(label_df['start_year'] <= year) & (label_df['end_year'] >= year)]
        return year_df.reset_index(drop=True)


    def __reduce_to_common__(self, labels_df):
        """
        Splits the labels DataFrame into two parts:
        1. Common labels that fall within the specified start and end years.
        2. Uncommon labels that do not fall within the specified range, and may be year specific.
        
        Args:
            labels_df (GeoDataFrame):   The labels DataFrame to reduce.
            start_year (int):           The starting year for the reduction.
            end_year (int):             The ending year for the reduction.
        
        Returns:
            GeoDataFrame:               The Reduced labels DataFrame.
        """
        common_df = labels_df[(labels_df['start_year'] <= self.start_year) & (labels_df['end_year'] >= self.end_year)]
        uncommon_df = labels_df[(labels_df['start_year'] > self.start_year) | (labels_df['end_year'] < self.end_year)]
        return common_df.reset_index(drop=True), uncommon_df.reset_index(drop=True)  


    @staticmethod
    def __split_year__(year, label_df):
        """
            Returns a DataFrame of where any label that overlaps a given year is returned.
        """

        year_df = label_df[(label_df['start_year'] <= year) & (label_df['end_year'] >= year)]
        # year_df['year'] = year
        return year_df.reset_index(drop=True)

    def __build_all_geometries__(self):
        """
            Builds a all Geometries DataFrame from the data labels
        """
        uniques = pd.DataFrame(columns=['file_name', '.geo'])
        for i in self.data_dir.index:
            df_dir = self.data_dir.loc[i, 'dir']
            # print(f"Loading Geometries {df_dir}...")

            df = pd.read_csv(df_dir)
            df = df[['file_name', '.geo']]
            print(f"Loaded {len(df)} geometries from {df_dir}.")

            df = df.dropna(subset=['file_name', '.geo'])
            df = df.reset_index(drop=True)
            
            uniques = pd.concat([uniques, df], ignore_index=True)
            uniques = uniques.drop_duplicates(subset=['file_name'])

            # no need to check all if we know the data IS FULL!
            if self.full_data:
                return uniques
            
        return uniques

    def __build_common_labels__(self):
        """
            Builds a common cluster from the common labels DataFrame. That ALL years share to avoid redundant processing.
        """


        uniuqes_gdf = self.__build_all_geometries__()
        uniuqes_gdf['geometry']  = uniuqes_gdf['.geo'].apply(lambda x: shape(json.loads(x)))
        uniuqes_gdf = gpd.GeoDataFrame(uniuqes_gdf, geometry='geometry', crs="EPSG:4326")

        cl_cluster = cl.Cluster(subregions, None, None, 1)
        cl_cluster.data = uniuqes_gdf.set_index("file_name")
        # cl_cluster.points = common_df.drop(columns=['start_year', 'end_year'])    
        cl_cluster.points = self.common_labels.drop(columns=['start_year', 'end_year'])

        cl_cluster.build_row_labels()

        # self.base_labels = cl_cluster.labels['file_name', 'label']
        self.base_labels = cl_cluster.labels
        return self.base_labels

    def instansiate_clusters(self, cluster_class, subregions, aoi=None, index_column="file_name"):
        """
            Instantiates clusters for each year in the data directory.
        """
        
        base_labelles = self.__build_common_labels__()
        
        cluster_list = []

        for year in self.data_dir['year']:
            data = pd.read_csv(self.data_dir.loc[data_dir['year'] == 2019, 'dir'].values[0])

            # (self, subregions, df_data, mapping, passes=6, aoi=None, index_column="file_name", points=gpd.GeoDataFrame()):
            cl_cluster = cluster_class(subregions=subregions, 
                                          df_data=data, 
                                          mapping=self.mapping, 
                                          points=self.common_labels,
                                          aoi=aoi,
                                          index_column=index_column,
                                          passes=self.passes)
            
            df_copy = base_labelles.copy()
            df_copy = df_copy.reset_index(drop=False)
            df_copy = df_copy[['file_name', 'label']]
            cl_cluster.load_labels_df(df=df_copy)
            
            cl_cluster.build_row_labels(additional_labels=self.yearly_labels_dict[year], update=True)

            cluster_list.append(cl_cluster)

        self.cluster_list = cluster_list

    def fit(self, save_state=""):
        """
            Fits all clusters in the cluster list.

            Args:
                save_state (str):         The prefix for the filename to save the state of the clusters. If empty, no state is saved.
        """

        for i, cl_cluster in enumerate(self.cluster_list):
            print(f"Fitted cluster for year {self.start_year + i} with {len(cl_cluster.labels)} labels.")

            cl_cluster.fit()

            if save_state is not "":
                cl_cluster.save_state(filname_prefix=save_state, filename_postfix=f"{self.start_year + i}_state")
        
        print(f"Fitted {len(self.cluster_list)} clusters.")

In [31]:
cluster_ts = ts_cluster(ts_point_labels=labels, data_dir=data_dir, mapping=mapping, start_year=2019, end_year=2024, passes=6)

cluster_ts.instansiate_clusters(cluster_class=cl.Cluster, subregions=subregions)

Loaded 375600 geometries from data/2019.csv.
progress: 1/4 labels processed.
progress: 2/4 labels processed.
progress: 3/4 labels processed.
progress: 4/4 labels processed.


In [ ]:
cluster_ts.base_labels

,.geo,geometry,label
file_name,,,
tile_43997486_34112017_43998137_34112113,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...","POLYGON ((43.99814 34.11211, 43.99749 34.11211...",None
tile_44104382_34112843_44105034_34112938,"{""type"":""Polygon"",""coordinates"":[[[44.10438435...","POLYGON ((44.10438 34.11293, 44.10438 34.11285...",None
tile_43998136_34112022_43999221_34112122,"{""type"":""Polygon"",""coordinates"":[[[43.99813739...","POLYGON ((43.99814 34.11211, 43.99814 34.11202...",None
tile_43999220_34112031_44000305_34112130,"{""type"":""Polygon"",""coordinates"":[[[43.99922097...","POLYGON ((43.99922 34.11212, 43.99922 34.11203...",None
tile_44000304_34112040_44001389_34112139,"{""type"":""Polygon"",""coordinates"":[[[44.00030452...","POLYGON ((44.00030 34.11213, 44.00030 34.11204...",None
...,...,...,...
tile_43910796_34410765_43911896_34411676,"{""type"":""Polygon"",""coordinates"":[[[43.91080588...","POLYGON ((43.91081 34.41077, 43.91189 34.41078...",None
tile_43910784_34411666_43911884_34412578,"{""type"":""Polygon"",""coordinates"":[[[43.91079693...","POLYGON ((43.91080 34.41167, 43.91189 34.41168...",None
tile_43910773_34412568_43911872_34413479,"{""type"":""Polygon"",""coordinates"":[[[43.91078360...","POLYGON ((43.91078 34.41257, 43.91187 34.41258...",None


In [33]:
cluster_ts.cluster_list[1].labels.head()

,system:index,0_B1,0_B11,0_B12,0_B2,0_B3,0_B4,0_B5,0_B6,0_B7,...,9_B3,9_B4,9_B5,9_B6,9_B7,9_B8,9_B8A,9_B9,file_name,.geo
0,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,769.925866,3322.198887,2644.624125,1128.620894,1753.336923,2010.568300,2634.353258,3302.266379,3455.818704,...,2541.523502,3334.350796,3648.235249,3713.545965,3826.787522,3871.171013,3920.213786,3837.507039,tile_43997486_34112017_43998137_34112113,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
1,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,298.404905,2157.473569,1582.494005,528.488556,831.356676,934.839237,1352.791826,1881.520708,2069.254223,...,801.668174,662.603379,1236.062398,2438.234332,2861.378801,3031.939292,3114.423978,2432.149682,tile_44104382_34112843_44105034_34112938,"{""type"":""Polygon"",""coordinates"":[[[44.10438435..."
2,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,528.454970,2495.875662,1713.649631,640.474810,1152.119767,1171.487171,1786.087047,3059.746338,3359.432014,...,1593.018742,2124.370565,2445.911989,2515.677113,2636.289053,2605.584472,2756.973423,3091.114321,tile_43998136_34112022_43999221_34112122,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
3,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,335.551290,2071.815893,1344.805779,429.478019,894.811249,770.613622,1463.251290,2839.394324,3180.367699,...,1394.102079,1854.067183,2060.790668,2105.297921,2202.409155,2225.273773,2299.559797,2382.651482,tile_43999220_34112031_44000305_34112130,"{""type"":""Polygon"",""coordinates"":[[[43.99922097..."
4,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,285.086948,1878.116894,1177.896530,406.417493,840.937113,721.051941,1302.629802,2778.296159,3145.126910,...,1386.387060,1850.829881,2059.086564,2108.126792,2208.764339,2278.053815,2330.043238,2451.489202,tile_44000304_34112040_44001389_34112139,"{""type"":""Polygon"",""coordinates"":[[[44.00030452..."


In [ ]:
cluster_ts.fit(save_state="temp_test_fit/test3")

In [ ]:
# get data_dir 2019
data_dir.loc[data_dir['year'] == 2019, 'dir'].values[0]

In [34]:
cluster_ts.common_labels

,label,geometry,start_year,end_year
0,Urban,POINT (43.88006 34.20145),2019,2024
1,Urban,POINT (43.88761 34.20457),2019,2024
2,Urban,POINT (43.85432 34.18292),2019,2024
3,Urban,POINT (43.90458 34.19124),2019,2024


In [ ]:
cluster_ts.cluster_list[0].labels.head()

In [ ]:
cluster = cl.Cluster(subregions=subregions, df_data=pd.read_csv(data_dir.loc[data_dir['year'] == 2019, 'dir'].values[0]), mapping=mapping, points=cluster_ts.common_labels, passes=6)
cluster.reload_state(filname_prefix="temp_test/test3", filename_postfix="_state")

cluster.labels.head()